In [1]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import os

d:\Develops\ML_PROJECT\NER-PROJECT\env_NER\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_path = "d:\\Develops\\ML_PROJECT\\NER-PROJECT\\Artifacts\\data\\train.tsv"
dev_path =  "d:\\Develops\\ML_PROJECT\\NER-PROJECT\\Artifacts\\data\\dev.tsv"
test_path = "d:\\Develops\\ML_PROJECT\\NER-PROJECT\\Artifacts\\data\\test.tsv"
pretrained_model = "dmis-lab/biobert-base-cased-v1.1"
max_length = 128

In [7]:
def build_label_map(tsv_file):
    labels = set()
    with open(tsv_file, encoding="utf-8") as f:
        for line in f:
            if line.strip() == "":
                continue
            _, tag = line.strip().split("\t")
            labels.add(tag)
    label2id = {label: idx for idx, label in enumerate(sorted(labels))}
    id2label = {v: k for k, v in label2id.items()}
    return label2id, id2label


In [8]:
label2id, id2label = build_label_map(train_path)

In [9]:
label2id, id2label

({'B-Chemical': 0, 'B-Disease': 1, 'I-Chemical': 2, 'I-Disease': 3, 'O': 4},
 {0: 'B-Chemical', 1: 'B-Disease', 2: 'I-Chemical', 3: 'I-Disease', 4: 'O'})

In [39]:
label_ids = [label2id.get("B-Chemical", label2id["O"])]

In [42]:
label_ids2= label_ids[:128]+[label2id['O']] * 10# * (128 - len(label_ids))

In [44]:
torch.tensor(label_ids2)

tensor([0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [37]:
label_ids1 = [label2id.get("B-Chemical", label2id["O"])]

In [38]:
label_ids1

[0]

In [14]:
class NerDataset(Dataset) :
    def __init__(self, filepath, label2id, tokenizer_name, max_length=128) :
        self.tokenizer = AutoTokenizer.from_pretrained(
            tokenizer_name,
            trust_remote_code=True,
            use_fast=True,
            add_prefix_space=True             
            )
        self.max_length = max_length
        self.label2id = label2id
        self.sentences, self.labels = self.load_tsv(filepath)

    
    def load_tsv(self, filepath) :
        setences, labels = [], []
        tokens, tags = [], []
        with open(filepath, encoding= "utf-8") as f :
            for line in f :
                print(line)

In [15]:
NerDataset(train_path, label2id, pretrained_model)

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-6906005a-02bb2acf0d4c1b4419e13763;c66733a1-4e22-405b-86db-d58f7fd30cd6)

Entry Not Found for url: https://huggingface.co/api/models/dmis-lab/biobert-base-cased-v1.1/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

In [ ]:
class NerDataset(Dataset):
    def __init__(self, filepath, label2id, tokenizer_name, max_length=128):
        self.tokenizer = AutoTokenizer.from_pretrained(
            tokenizer_name,
            trust_remote_code=True,
            use_fast=True,
            add_prefix_space=True  # Helps with word-level tokenization
        )
        self.max_length = max_length
        self.label2id = label2id
        self.sentences, self.labels = self.load_tsv(filepath)

In [12]:
class NerDataset(Dataset):
    def __init__(self, filepath, label2id, tokenizer_name, max_length=128):
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(
                tokenizer_name,
                trust_remote_code=True,
                use_fast=True
            )
        except Exception as e:
            print(f"Error loading tokenizer: {e}")
            print("Falling back to basic BERT tokenizer")
            self.tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
        
        self.max_length = max_length
        self.label2id = label2id
        self.sentences, self.labels = self.load_tsv(filepath)

In [16]:
from transformers import AutoTokenizer
import os

# Download tokenizer files locally
model_name = "dmis-lab/biobert-base-cased-v1.1"
cache_dir = "d:\\cached_models"
os.makedirs(cache_dir, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=cache_dir,
    trust_remote_code=True,
    use_fast=True
)

d:\Develops\ML_PROJECT\NER-PROJECT\env_NER\Lib\site-packages\huggingface_hub\file_download.py:120: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\cached_models\models--dmis-lab--biobert-base-cased-v1.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  except OSError:


RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-690600b5-29eaa8826b057f440083f959;9293ae4b-6d83-4ca3-87ea-4c26d70b45b3)

Entry Not Found for url: https://huggingface.co/api/models/dmis-lab/biobert-base-cased-v1.1/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

In [17]:
import os
from transformers import AutoTokenizer

def initialize_tokenizer(model_name, cache_dir=None):
    """
    Safely initialize a tokenizer with fallback options
    """
    try:
        # First attempt: Basic initialization
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            use_fast=True,
            trust_remote_code=True,
            cache_dir=cache_dir
        )
    except Exception as primary_error:
        print(f"Primary tokenizer loading failed: {primary_error}")
        try:
            # Fallback: Try without chat templates
            tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                use_fast=True,
                trust_remote_code=True,
                cache_dir=cache_dir,
                model_max_length=512,  # Explicit max length
                padding_side='right',  # Explicit padding side
                use_auth_token=None    # Explicit auth setting
            )
        except Exception as fallback_error:
            print(f"Fallback tokenizer loading failed: {fallback_error}")
            # Final fallback: Use base BERT tokenizer
            tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
            
    return tokenizer

# Usage
cache_dir = "d:/cached_models"
os.makedirs(cache_dir, exist_ok=True)
model_name = "dmis-lab/biobert-base-cased-v1.1"

tokenizer = initialize_tokenizer(model_name, cache_dir)

Primary tokenizer loading failed: 404 Client Error. (Request ID: Root=1-6906011a-73d84a62358d9eed325d5274;b39e6be2-6d41-40d5-b2df-a1f0a3875468)

Entry Not Found for url: https://huggingface.co/api/models/dmis-lab/biobert-base-cased-v1.1/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"
Fallback tokenizer loading failed: 404 Client Error. (Request ID: Root=1-6906011b-6144e49365a6d3a2390227a1;f8e2d4bb-eb39-4a28-a11c-86101cad2e63)

Entry Not Found for url: https://huggingface.co/api/models/dmis-lab/biobert-base-cased-v1.1/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"


RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-6906011c-1e8429cc04d3a95a3949c6a4;f291ff7a-89a9-467c-8a17-03990e7bdcf9)

Entry Not Found for url: https://huggingface.co/api/models/google-bert/bert-base-cased/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"

In [17]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import os

class NERDataset(Dataset):
    def __init__(self, filepath, label2id, tokenizer_name, max_length=128):
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_length = max_length
        self.label2id = label2id
        self.sentences, self.labels = self.load_tsv(filepath)
        print(self.labels)
    def load_tsv(self, filepath):
        sentences, labels = [], []
        tokens, tags = [], []
        with open(filepath, encoding="utf-8") as f:
            for line in f:
                if line.strip() == "":
                    #print(f"line {line.strip()}")
                    if tokens :
                        # print("line {line}")
                        # print("tokens {tokens}")
                        sentences.append(tokens)
                        labels.append(tags)
                        # print(f"sentence {sentences}")
                        # print(f"labels {labels}")
                        tokens, tags = [], []
                else :
                    token, tag = line.strip().split("\t")
                    tokens.append(token)
                    tags.append(tag)
                    # print(f"token {token}")
                    # print(f"tag {tag}")
        return sentences, labels

    def __getitem__(self, idx):
        tokens = self.sentences[idx]
        labels = self.labels[idx]

        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        label_ids = [self.label2id.get(l, self.label2id["O"]) for l in labels]
        label_ids = label_ids[:self.max_length] + [self.label2id["O"]] * (self.max_length - len(label_ids))
        encoding["labels"] = torch.tensor(label_ids)
        return {k: v.squeeze(0) for k, v in encoding.items()}

In [18]:
nerdataset_object = NERDataset(train_path, label2id, pretrained_model) #.load_tsv(train_path)

[['I-Chemical', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'I-Disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'I-Disease', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Disease', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Chemical', 'O', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['I-Chemical', 'I-Dis

In [22]:
label2id

{'B-Chemical': 0, 'B-Disease': 1, 'I-Chemical': 2, 'I-Disease': 3, 'O': 4}

In [26]:
# Python
for label_sequence in nerdataset_object.labels:
    label_ids = [label2id.get(label, label2id['O']) for label in label_sequence]
    print(label_ids)  # or do something with the converted IDs

[2, 4, 4, 4, 4, 4, 2, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
[2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2]
[2, 4, 4, 4, 4, 3, 4, 4, 4, 2, 2, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4

In [ ]:
 def __getitem__(self, idx):
        tokens = self.sentences[idx]
        labels = self.labels[idx]

        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        label_ids = [self.label2id.get(l, self.label2id["O"]) for l in labels]
        label_ids = label_ids[:self.max_length] + [self.label2id["O"]] * (self.max_length - len(label_ids))
        encoding["labels"] = torch.tensor(label_ids)
        return {k: v.squeeze(0) for k, v in encoding.items()}

In [1]:
a = "joti"